In [9]:
import numpy as np
import nidaqmx
from nidaqmx.constants import Edge, LineGrouping, AcquisitionType, Signal
from nidaqmx.stream_writers import AnalogMultiChannelWriter, DigitalSingleChannelWriter
import time

In [23]:
# Sweep parameters
voltages_ao1 = np.linspace(-2, 2, 100)
voltages_ao2 = np.linspace(-2, 2, 100)
v_min, v_max = -2.0, 2.0
sample_rate = 100
num_samples = len(voltages_ao1)

# Analog data (2 channels x N samples)
ao_data = np.vstack([voltages_ao1, voltages_ao2])

with nidaqmx.Task() as ao_task:
    # Add AO channels
    ao_task.ao_channels.add_ao_voltage_chan("Dev1/ao1", min_val=v_min, max_val=v_max)
    ao_task.ao_channels.add_ao_voltage_chan("Dev1/ao2", min_val=v_min, max_val=v_max)

    # Configure AO sample clock
    ao_task.timing.cfg_samp_clk_timing(rate=sample_rate,
                                       sample_mode=AcquisitionType.FINITE,
                                       samps_per_chan=num_samples)

    # ✅ Export the AO Sample Clock to PFI0
    ao_task.export_signals.sample_clock_output_term = "Dev1/PFI0"

    # Write data
    AnalogMultiChannelWriter(ao_task.out_stream).write_many_sample(ao_data)

    # Start task
    ao_task.start()
    ao_task.wait_until_done(timeout=10)
print("Sweep complete. Trigger pulses sent via AO Sample Clock on PFI0.")

with nidaqmx.Task() as task:
    # Add analog output channels with voltage range limits
    task.ao_channels.add_ao_voltage_chan("Dev1/ao1", min_val=v_min, max_val=v_max)
    task.ao_channels.add_ao_voltage_chan("Dev1/ao2", min_val=v_min, max_val=v_max)
    task.write([0, 0], auto_start=True)

Sweep complete. Trigger pulses sent via AO Sample Clock on PFI0.
